In [1]:
%%capture
pip install numpy pandas

In [2]:
import numpy as np
import pandas as pd

In [3]:
cohort_users = pd.read_csv('../sources/cohort_users.csv')

In [4]:
auth = pd.read_csv('../sources/auth_user.csv')

In [5]:
upcoming_cohorts = pd.read_csv('../output/upcoming_cohorts.csv')

In [6]:
upcoming_cohorts_slugs = upcoming_cohorts[['id', 'slug']]

In [7]:
upcoming_cohorts_slugs

,id,slug
0,501,spain-fs-pt-enero
1,384,latam-pt-ml-2
2,473,latam-pt-20
3,470,latam-pt-17
4,504,spain-fs-pt-abril
5,403,miami-50
6,471,latam-pt-18
7,505,spain-fs-pt-mayo
8,479,latam-pt-26
9,478,latam-pt-25


In [8]:
cohort_users = cohort_users.merge(upcoming_cohorts_slugs, left_on='cohort_id', right_on='id', how='left')
cohort_users = cohort_users.rename(columns={'id_x':'id'})
cohort_users = cohort_users.drop(columns=['id_y'])

In [9]:
cohort_users.columns

Index(['id', 'role', 'finantial_status', 'educational_status', 'created_at',
       'updated_at', 'cohort_id', 'user_id', 'watching', 'slug'],
      dtype='object')

In [10]:
cohort_users_auth = cohort_users.merge(auth, left_on='user_id', right_on='id', how='left')

In [11]:
cohort_users_auth = cohort_users_auth[cohort_users_auth['role'].str.lower()=='student']

In [12]:
cohort_users_auth.columns

Index(['id_x', 'role', 'finantial_status', 'educational_status', 'created_at',
       'updated_at', 'cohort_id', 'user_id', 'watching', 'slug', 'id_y',
       'first_name', 'last_name', 'email', 'date_joined'],
      dtype='object')

In [13]:
confirmed = cohort_users_auth.loc[cohort_users_auth['slug'].notnull()]

In [18]:
confirmed

,id_x,role,finantial_status,educational_status,created_at,updated_at,cohort_id,user_id,watching,slug,id_y,first_name,last_name,email,date_joined


In [15]:
confirmed = confirmed[['slug', 'email']]

In [14]:
confirmed['unconfirmed'] = 0
confirmed['confirmed'] = 1

In [15]:
confirmed

,id_x,role,finantial_status,educational_status,created_at,updated_at,cohort_id,user_id,watching,slug,id_y,first_name,last_name,email,date_joined,unconfirmed,confirmed


In [22]:
fake_confirmed = {'slug':['caracas-pt-41'], 'email':['cdlacostesalazar@gmail.com'], 'unconfirmed':[0], 'confirmed':[1]}

In [23]:
fake_df_confirmed = pd.DataFrame(fake_confirmed)

In [24]:
confirmed = pd.concat([confirmed, fake_df_confirmed])

In [25]:
confirmed

,slug,email,unconfirmed,confirmed
0,caracas-pt-41,cdlacostesalazar@gmail.com,0,1


In [26]:
# Concatenate the two dataframes
concatenated_df = pd.concat([confirmed, unconfirmed])

In [27]:
# Group the rows by the 'slug' and 'email' columns, and sum the 'confirmed' and 'unconfirmed' columns
upcoming_cohorts_users = concatenated_df.groupby(['slug', 'email']).sum().reset_index()

In [28]:
upcoming_cohorts_users

,slug,email,unconfirmed,confirmed
0,caracas-pt-41,cdlacostesalazar@gmail.com,1,1
1,cohort-34,justjesi2010@gmail.com,1,0
2,full-stack-21,wambuangui0746@gmail.com,1,0
3,latam-pt-ml-2,marcelohauser2013@gmail.com,1,0
4,madrid-pt-13,02sergioconde@gmail.com,1,0
...,...,...,...,...
5586,madrid-pt-13,ziyad.saber@alumnos.upm.es,1,0
5587,madrid-pt-13,ztxuzz@gmail.com,1,0
5588,madrid-pt-13,zubererross@gmail.com,1,0
5589,madrid-pt-13,zuberlye@gmail.com,1,0


In [29]:
df_unconfirmed = upcoming_cohorts_users[upcoming_cohorts_users['unconfirmed']==1]

In [30]:
df_unconfirmed = pd.merge(df_unconfirmed, form_entry, left_on=['slug', 'email'], right_on=['ac_expected_cohort', 'email'])

In [31]:
df_unconfirmed

,slug,email,unconfirmed,confirmed,id,first_name,last_name,phone,course,client_comments,...,ac_expected_cohort,current_download,lead_generation_app_id,utm_content,storage_status_text,custom_fields,sex,utm_placement,utm_plan,utm_term
0,caracas-pt-41,cdlacostesalazar@gmail.com,1,1,58974,Carlos D'Lacoste,NaN,5.842429e+11,full-stack,NaN,...,caracas-pt-41,NaN,NaN,121425136982,NaN,NaN,NaN,NaN,NaN,NaN
1,cohort-34,justjesi2010@gmail.com,1,0,22363,Jessica Allen,NaN,1.405410e+10,full-stack,NaN,...,cohort-34,NaN,NaN,129223198448,NaN,NaN,NaN,NaN,NaN,NaN
2,full-stack-21,wambuangui0746@gmail.com,1,0,14354,Wambua,NaN,7.462385e+08,software-engineering,NaN,...,full-stack-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,latam-pt-ml-2,marcelohauser2013@gmail.com,1,0,8975,Marcelo Hauser,NaN,9.582728e+08,full-stack,NaN,...,latam-pt-ml-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,madrid-pt-13,02sergioconde@gmail.com,1,0,28779,Sergio Conde Alcalde,NaN,3.468109e+10,full-stack-ft,NaN,...,madrid-pt-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6992,madrid-pt-13,ztxuzz@gmail.com,1,0,23700,Jesús Rico,NaN,3.468035e+10,full-stack,NaN,...,madrid-pt-13,NaN,NaN,132081395476,NaN,NaN,NaN,NaN,NaN,NaN
6993,madrid-pt-13,zubererross@gmail.com,1,0,36205,Zoubair Errossafi Chakroun,NaN,3.464085e+10,full-stack,NaN,...,madrid-pt-13,NaN,NaN,134614741199,NaN,NaN,NaN,NaN,NaN,NaN
6994,madrid-pt-13,zuberlye@gmail.com,1,0,26451,Zuberly Sanchez,NaN,3.464203e+10,full-stack,NaN,...,madrid-pt-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6995,madrid-pt-13,zuberlye@gmail.com,1,0,26452,NaN,NaN,NaN,NaN,NaN,...,madrid-pt-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df_unconfirmed.columns

Index(['slug', 'email', 'unconfirmed', 'confirmed', 'id', 'first_name',
       'last_name', 'phone', 'course', 'client_comments', 'language',
       'utm_url', 'utm_medium', 'utm_campaign', 'street_address', 'country',
       'city', 'latitude', 'longitude', 'state', 'zip_code', 'storage_status',
       'created_at', 'updated_at', 'contact_id', 'lead_type', 'location',
       'referral_key', 'automations', 'tags', 'gclid', 'browser_lang',
       'fb_ad_id', 'fb_adgroup_id', 'fb_form_id', 'fb_leadgen_id',
       'fb_page_id', 'utm_source', 'academy_id', 'deal_status', 'sentiment',
       'ac_contact_id', 'ac_deal_id', 'user_id', 'won_at',
       'ac_expected_cohort', 'current_download', 'lead_generation_app_id',
       'utm_content', 'storage_status_text', 'custom_fields', 'sex',
       'utm_placement', 'utm_plan', 'utm_term'],
      dtype='object')

In [33]:
df_unconfirmed['datetime_update'] = pd.to_datetime(df_unconfirmed['updated_at'])

In [34]:
#grouped_unconfirmed = df_unconfirmed.groupby(['slug', 'email']).agg({'datetime_update': 'max', 'location': 'last', 'ac_deal_id':'last', 'ac_deal_owner_full_name':'last', 'created_at_y': 'last', 'campus_check':'last'})
grouped_unconfirmed = df_unconfirmed.groupby(['slug', 'email']).agg({'datetime_update': 'max', 'location': 'last', 'ac_deal_id':'last'})
grouped_unconfirmed = grouped_unconfirmed.reset_index()

In [35]:
upcoming_cohorts_users = upcoming_cohorts_users.merge(grouped_unconfirmed, on='email', how='left').drop(columns=['slug_y', 'datetime_update'])

In [36]:
upcoming_cohorts_users = upcoming_cohorts_users.rename(columns={'slug_x':'slug'})
# upcoming_cohorts_users = upcoming_cohorts_users.rename(columns={'slug_x':'slug', 'created_at_y':'form_created_at', 'location':'expected_location', 'ac_deal_id':'deal_id', 'ac_deal_owner_full_name':'deal_owner'})

In [37]:
upcoming_cohorts_users

,slug,email,unconfirmed,confirmed,location,ac_deal_id
0,caracas-pt-41,cdlacostesalazar@gmail.com,1,1,test_added_location,74497.0
1,cohort-34,justjesi2010@gmail.com,1,0,orlando,34644.0
2,full-stack-21,wambuangui0746@gmail.com,1,0,downtown-miami,27957.0
3,latam-pt-ml-2,marcelohauser2013@gmail.com,1,0,test_added_location_2,18823.0
4,madrid-pt-13,02sergioconde@gmail.com,1,0,madrid-spain,29045.0
...,...,...,...,...,...,...
5586,madrid-pt-13,ziyad.saber@alumnos.upm.es,1,0,madrid-spain,78810.0
5587,madrid-pt-13,ztxuzz@gmail.com,1,0,madrid-spain,38161.0
5588,madrid-pt-13,zubererross@gmail.com,1,0,madrid-spain,54084.0
5589,madrid-pt-13,zuberlye@gmail.com,1,0,madrid-spain,NaN


In [38]:
df_confirmed = upcoming_cohorts_users[upcoming_cohorts_users['confirmed']==1]

In [39]:
df_confirmed.columns

Index(['slug', 'email', 'unconfirmed', 'confirmed', 'location', 'ac_deal_id'], dtype='object')

In [40]:
df_confirmed = pd.merge(df_confirmed, cohort_users_auth, on=['slug', 'email'])

In [41]:
df_confirmed['datetime_update'] = pd.to_datetime(df_confirmed['updated_at'])

In [42]:
df_confirmed.columns

Index(['slug', 'email', 'unconfirmed', 'confirmed', 'location', 'ac_deal_id',
       'id_x', 'role', 'finantial_status', 'educational_status', 'created_at',
       'updated_at', 'cohort_id', 'user_id', 'watching', 'id_y', 'first_name',
       'last_name', 'date_joined', 'datetime_update'],
      dtype='object')

In [43]:
grouped_confirmed = df_confirmed.groupby(['slug', 'email']).agg({'datetime_update': 'max', 'educational_status': 'last', 'created_at': 'last'})
grouped_confirmed = grouped_confirmed.reset_index()

In [44]:
upcoming_cohorts_users = upcoming_cohorts_users.merge(grouped_confirmed, on='email', how='left')

In [45]:
upcoming_cohorts_users = upcoming_cohorts_users.rename(columns={'slug_x':'slug', 'created_at':'inscription_created_at'}).drop(columns=['slug_y'])

In [46]:
upcoming_cohorts_users

,slug,email,unconfirmed,confirmed,location,ac_deal_id,datetime_update,educational_status,inscription_created_at
0,caracas-pt-41,cdlacostesalazar@gmail.com,1,1,test_added_location,74497.0,NaT,NaN,NaN
1,cohort-34,justjesi2010@gmail.com,1,0,orlando,34644.0,NaT,NaN,NaN
2,full-stack-21,wambuangui0746@gmail.com,1,0,downtown-miami,27957.0,NaT,NaN,NaN
3,latam-pt-ml-2,marcelohauser2013@gmail.com,1,0,test_added_location_2,18823.0,NaT,NaN,NaN
4,madrid-pt-13,02sergioconde@gmail.com,1,0,madrid-spain,29045.0,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...
5586,madrid-pt-13,ziyad.saber@alumnos.upm.es,1,0,madrid-spain,78810.0,NaT,NaN,NaN
5587,madrid-pt-13,ztxuzz@gmail.com,1,0,madrid-spain,38161.0,NaT,NaN,NaN
5588,madrid-pt-13,zubererross@gmail.com,1,0,madrid-spain,54084.0,NaT,NaN,NaN
5589,madrid-pt-13,zuberlye@gmail.com,1,0,madrid-spain,NaN,NaT,NaN,NaN


In [47]:
upcoming_cohorts_users = upcoming_cohorts.merge(upcoming_cohorts_users, on='slug', how='left')

In [48]:
upcoming_cohorts_users

,Unnamed: 0,id,slug,name,kickoff_date,ending_date,current_day,stage,language,created_at,...,campus,datetime_kickoff,email,unconfirmed,confirmed,location,ac_deal_id,datetime_update,educational_status,inscription_created_at
0,124,501,spain-fs-pt-enero,Spain FS PT Enero,2023-06-06 14:16:00,2023-10-10 15:16:00,0,INACTIVE,es,2022-12-09 15:17:36.465700,...,4Geeks Academy Madrid,2023-06-06 14:16:00,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1,191,384,latam-pt-ml-2,latam-pt-ml-2,2023-03-20 19:55:00,2023-07-21 19:55:00,0,INACTIVE,es,2022-06-15 19:56:41.393446,...,4Geeks Academy Latam,2023-03-20 19:55:00,marcelohauser2013@gmail.com,1.0,0.0,test_added_location_2,18823.0,NaT,NaN,NaN
2,221,473,latam-pt-20,latam-pt-20,2023-06-19 18:16:00,2023-10-20 18:16:00,0,INACTIVE,es,2022-10-18 18:17:28.070083,...,4Geeks Academy Latam,2023-06-19 18:16:00,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
3,222,470,latam-pt-17,latam-pt-17,2023-03-13 18:12:00,2023-07-17 18:12:00,0,INACTIVE,es,2022-10-18 18:14:09.717460,...,4Geeks Academy Latam,2023-03-13 18:12:00,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
4,262,504,spain-fs-pt-abril,Spain FS PT Abril,2023-04-10 15:24:00,2023-08-14 15:24:00,0,INACTIVE,es,2022-12-09 15:26:39.048445,...,4Geeks Academy Madrid,2023-04-10 15:24:00,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
5,266,403,miami-50,Miami 50,2023-03-20 14:55:00,2023-07-21 14:55:00,0,INACTIVE,en,2022-08-11 14:55:57.057573,...,4Geeks Academy Miami,2023-03-20 14:55:00,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
6,267,471,latam-pt-18,latam-pt-18,2023-04-17 18:14:00,2023-08-18 18:14:00,0,INACTIVE,es,2022-10-18 18:15:23.162442,...,4Geeks Academy Latam,2023-04-17 18:14:00,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
7,293,505,spain-fs-pt-mayo,Spain FS PT Mayo,2023-05-02 15:26:00,2023-09-05 15:26:00,0,INACTIVE,es,2022-12-09 15:27:49.384144,...,4Geeks Academy Madrid,2023-05-02 15:26:00,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
8,308,479,latam-pt-26,latam-pt-26,2023-12-11 18:36:00,2024-04-26 18:36:00,0,INACTIVE,es,2022-10-18 18:36:56.343456,...,4Geeks Academy Latam,2023-12-11 18:36:00,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
9,311,478,latam-pt-25,latam-pt-25,2023-11-13 18:33:00,2024-03-29 18:33:00,0,INACTIVE,es,2022-10-18 18:34:52.895475,...,4Geeks Academy Latam,2023-11-13 18:33:00,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
